In [1]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import os
from pathlib import Path
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import json

load_dotenv()

PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT")).resolve()  # type: ignore

def ema(values, alpha=0.98):
    """Exponential Moving Average smoothing"""
    smoothed = []
    m = None
    for v in values:
        m = v if m is None else alpha * m + (1 - alpha) * v
        smoothed.append(m)
    return smoothed

def draw_one_scalar(event_accs, tag, graph_dir, names=None):
    plt.figure(figsize=(10, 5))
    labels = names if names is not None else [f'Log {i+1}' for i in range(len(event_accs))]
    for i, event_acc in enumerate(event_accs):
        scalar_events = event_acc.Scalars(tag)
        steps = [e.step for e in scalar_events]
        values = [e.value for e in scalar_events]
        plt.plot(steps, values, color=f"C{i}", alpha=0.1)
    for i, event_acc in enumerate(event_accs):
        scalar_events = event_acc.Scalars(tag)
        steps = [e.step for e in scalar_events]
        values = [e.value for e in scalar_events]
        smoothed_values = ema(values)
        plt.plot(steps, smoothed_values, label=labels[i], color=f"C{i}")
    plt.xlabel('Step')
    plt.ylabel(tag)
    plt.title(f'Scalar: {tag}')
    plt.legend()
    plt.grid(True)
    plt.savefig(graph_dir / f"{tag.replace('/', '_')}.png", dpi=300)
    plt.close()

In [2]:
log_root = PROJECT_ROOT / "logs" / "experiment1"
log_dirs = []
event_accs = []
names = []

with open(log_root / "dir_to_name.json", "r") as f:
    dir_to_name = json.load(f)

for sub in log_root.iterdir():
    if sub.is_dir():
        if dir_to_name.get(sub.name, sub.name) == "ignore":
            continue
        log_dirs.append(sub)
        event_acc = EventAccumulator(str(sub))
        event_acc.Reload()
        event_accs.append(event_acc)
        names.append(dir_to_name.get(sub.name, sub.name))

In [3]:
graph_dir = PROJECT_ROOT / "graphs" / "comparisons"
os.makedirs(graph_dir, exist_ok=True)

In [4]:
# only common tags
scalars = set(event_accs[0].Tags()['scalars'])
for event_acc in event_accs[1:]:
    scalars.intersection_update(set(event_acc.Tags()['scalars']))

for tag in scalars:
    draw_one_scalar(event_accs, tag, graph_dir, names)
    print(f"Saved graph for {tag}")

Saved graph for train/epoch
Saved graph for train/rewards/accuracies
Saved graph for train/logps/chosen
Saved graph for train/learning_rate
Saved graph for train/rewards/rejected
Saved graph for train/rewards/chosen
Saved graph for train/loss
Saved graph for train/grad_norm
Saved graph for train/rewards/margins
Saved graph for train/logits/chosen
Saved graph for train/logps/rejected
Saved graph for train/logits/rejected
